In [9]:
import json
import statistics
from openai import OpenAI
import dotenv 
import os
dotenv.load_dotenv()
import pandas as pd

In [13]:
entities = pd.read_parquet("GraphRAG2\output\entities.parquet")
relations = pd.read_parquet("GraphRAG2\\output\\relationships.parquet")
communities = pd.read_parquet("GraphRAG2\\output\\communities.parquet")
comm_reports= pd.read_parquet("GraphRAG2\\output\\community_reports.parquet")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2248 entries, 0 to 2247
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 2248 non-null   object 
 1   human_readable_id  2248 non-null   int64  
 2   title              2248 non-null   object 
 3   type               2248 non-null   object 
 4   description        2248 non-null   object 
 5   text_unit_ids      2248 non-null   object 
 6   frequency          2248 non-null   int64  
 7   degree             2248 non-null   int32  
 8   x                  1851 non-null   float64
 9   y                  1851 non-null   float64
dtypes: float64(2), int32(1), int64(2), object(5)
memory usage: 167.0+ KB


In [15]:
relations

,id,human_readable_id,source,target,description,weight,combined_degree,text_unit_ids
0,740ea2d1-dec3-48ce-83f3-18da712e79a5,0,CODE GENERAL DES IMPOTS,LOI DE FINANCES N° 43-06,Le Code Général des Impôts et la Loi de Financ...,23.0,49,[bebf00d607f13edc091bc9048e950a23998255e5c6899...
1,dc9d2935-5f9e-4539-bc46-ea6cc343a330,1,CODE GENERAL DES IMPOTS,LOI-CADRE N° 69-19,La loi-cadre n° 69-19 a introduit des réformes...,7.0,34,[bebf00d607f13edc091bc9048e950a23998255e5c6899...
2,c29be282-68d8-48ca-a5c5-dd642479f016,2,CODE GENERAL DES IMPOTS,TAXE SUR LA VALEUR AJOUTEE,La Taxe sur la Valeur Ajoutée est intégrée dan...,6.0,47,[bebf00d607f13edc091bc9048e950a23998255e5c6899...
3,5b2d5d10-8906-4762-a185-b98c58bc33b3,3,CODE GENERAL DES IMPOTS,IMPOT SUR LE REVENU,L'Impôt sur le Revenu est intégré dans le Code...,6.0,35,[bebf00d607f13edc091bc9048e950a23998255e5c6899...
4,df0b4ce7-f925-4a25-81ae-a4e785ad4a8c,4,CODE GENERAL DES IMPOTS,IMPOT SUR LES SOCIETES,L'Impôt sur les Sociétés est intégré dans le C...,1.0,34,[bebf00d607f13edc091bc9048e950a23998255e5c6899...
...,...,...,...,...,...,...,...,...
3059,dea84b95-0a9b-4e89-bd8e-3e2352bac419,3059,TAUX D'INTÉRÊT,ANNÉE 2023,Les taux d'intérêt de l'année 2023 sont import...,8.0,10,[ca948aaaa96f24aac91238631d5c8c1ff677d6f149190...
3060,45e7c3ac-772a-46bd-9c04-673cdfee0601,3060,TAUX D'INTÉRÊT,ANNÉE 2024,Les taux d'intérêt projetés pour l'année 2024 ...,1.0,7,[ca948aaaa96f24aac91238631d5c8c1ff677d6f149190...
3061,5f2500dd-c01a-4934-adbd-8de7cd6ddc43,3061,TAUX D'INTÉRÊT,2022,Les taux d'intérêt de l'année 2022 ont été enr...,8.0,8,[ca948aaaa96f24aac91238631d5c8c1ff677d6f149190...
3062,9e442ccf-db83-4233-be35-0639baf00c37,3062,TAUX D'INTÉRÊT,2023,Les taux d'intérêt de l'année 2023 sont import...,8.0,8,[ca948aaaa96f24aac91238631d5c8c1ff677d6f149190...


In [19]:
communities

array([0, 1, 2, 3, 4])

In [17]:
comm_reports

,id,human_readable_id,community,level,parent,children,title,summary,full_content,rank,rating_explanation,findings,full_content_json,period,size
0,e93270e186e946238db8f7e57cd9001a,294,294,4,286,[],Analyse du Code Général des Impôts au Maroc,Ce rapport examine la structure et les implica...,# Analyse du Code Général des Impôts au Maroc\...,9.5,Le rapport est hautement significatif pour com...,[{'explanation': 'Le Code Général des Impôts (...,"{\n ""title"": ""Analyse du Code Général des I...",2025-12-16,152
1,8d72378c3be24332ab696d8bd7c1954e,295,295,4,286,[],Impact de l'Annulation des Amendes et des Date...,La communauté se concentre sur l'impact de l'a...,# Impact de l'Annulation des Amendes et des Da...,8.0,Le rapport est très pertinent car il traite de...,"[{'explanation': 'L'annulation des amendes, pé...","{\n ""title"": ""Impact de l'Annulation des Am...",2025-12-16,2
2,e3cac7f922354831a1e865c7d102e680,286,286,3,163,"[294, 295]",Analyse du Code Général des Impôts au Maroc,Ce rapport examine la structure et les implica...,# Analyse du Code Général des Impôts au Maroc\...,9.0,Le rapport est hautement pertinent pour compre...,[{'explanation': 'Le Code Général des Impôts (...,"{\n ""title"": ""Analyse du Code Général des I...",2025-12-16,154
3,8c9d0b5d7b79452cb133dce374c87ca1,287,287,3,163,[],Exonération de la TVA et Compte Matières,La communauté se concentre sur l'article 92I-5...,# Exonération de la TVA et Compte Matières\n\n...,9.0,Le rapport est très pertinent car il traite de...,[{'explanation': 'L'article 92I-54° du code gé...,"{\n ""title"": ""Exonération de la TVA et Comp...",2025-12-16,2
4,333fd9d3805047f2afa6433d3e9e4792,288,288,3,167,[],Analyse des Implications Fiscales du Code Géné...,Cette communauté est centrée sur le Code Génér...,# Analyse des Implications Fiscales du Code Gé...,9.0,Le rapport est hautement pertinent pour compre...,[{'explanation': 'L'article 60 du Code général...,"{\n ""title"": ""Analyse des Implications Fisc...",2025-12-16,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,88b43442d0fb42618960f3369954a72b,25,25,0,-1,"[154, 155]",Analyse de la Loi de Finances et des Articles ...,La communauté se concentre sur la Loi de Finan...,# Analyse de la Loi de Finances et des Article...,9.0,Le rapport est très pertinent car il traite de...,[{'explanation': 'La Loi de Finances est un te...,"{\n ""title"": ""Analyse de la Loi de Finances...",2025-12-16,14
292,9cc58a29dc024ddda59a44d147c3828a,26,26,0,-1,[],Décret N° 2.21.315 et Code Général des Impôts,La communauté se concentre sur le décret n° 2....,# Décret N° 2.21.315 et Code Général des Impôt...,9.0,Le rapport est très pertinent car il traite de...,[{'explanation': 'Le décret n° 2.21.315 est es...,"{\n ""title"": ""Décret N° 2.21.315 et Code Gé...",2025-12-16,5
293,72d234cf57ae49038a049ef5272dc1d6,27,27,0,-1,"[156, 157, 158, 159]",Fondations et Lois de Lutte Contre les Maladie...,La communauté se compose de plusieurs fondatio...,# Fondations et Lois de Lutte Contre les Malad...,9.0,Le rapport est très pertinent pour comprendre ...,[{'explanation': 'La Fondation Mohammed VI a é...,"{\n ""title"": ""Fondations et Lois de Lutte C...",2025-12-16,14
294,e8ee5ff587cd4f738134fa69748f35b9,28,28,0,-1,[],Analyse de la Loi N° 21.90 et des Concessionna...,La communauté se concentre sur la Loi N° 21.90...,# Analyse de la Loi N° 21.90 et des Concession...,9.0,Le rapport est très pertinent car il traite de...,[{'explanation': 'La Loi N° 21.90 est fondamen...,"{\n ""title"": ""Analyse de la Loi N° 21.90 et...",2025-12-16,4


In [ ]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [3]:
df=pd.read_csv("questions.csv",sep=";")

In [28]:
client = OpenAI(api_key=os.getenv("GRAPHRAG_API_KEY"))

In [29]:
CRITERIA = {
"comprehensiveness": """How much detail does the answer provide to cover all the aspects and details of the
question? A comprehensive answer should be thorough and complete, without being redundant or irrelevant.
For example, if the question is ’What are the benefits and drawbacks of nuclear energy?’, a comprehensive
answer would provide both the positive and negative aspects of nuclear energy, such as its efficiency,
environmental impact, safety, cost, etc. A comprehensive answer should not leave out any important points
or provide irrelevant information. For example, an incomplete answer would only provide the benefits of
nuclear energy without describing the drawbacks, or a redundant answer would repeat the same information
multiple times.""",
"diversity": """How varied and rich is the answer in providing different perspectives and insights
on the question? A diverse answer should be multi-faceted and multi-dimensional, offering different
viewpoints and angles on the question. For example, if the question is ’What are the causes and effects
of climate change?’, a diverse answer would provide different causes and effects of climate change, such
as greenhouse gas emissions, deforestation, natural disasters, biodiversity loss, etc. A diverse answer
should also provide different sources and evidence to support the answer. For example, a single-source
answer would only cite one source or evidence, or a biased answer would only provide one perspective or
opinion.""",
"directness": """How specifically and clearly does the answer address the question? A direct answer should
provide a clear and concise answer to the question. For example, if the question is ’What is the capital
of France?’, a direct answer would be ’Paris’. A direct answer should not provide any irrelevant or
unnecessary information that does not answer the question. For example, an indirect answer would be ’The
capital of France is located on the river Seine’."""
}

In [30]:
JUDGE_SYSTEM_PROMPT = "You are a helpful assistant responsible for grading two answers to a question that are provided by two different people."

JUDGE_USER_TEMPLATE = """
---Goal---
Given a question and two answers (Answer 1 and Answer 2), assess which answer is better according to the following measure:
{criterion_def}

Your assessment should include two parts:
- Winner: either 1 (if Answer 1 is better) and 2 (if Answer 2 is better) or 0 if they are fundamentally similar and the differences are immaterial.
- Reasoning: a short explanation of why you chose the winner with respect to the measure described above.

Format your response as a JSON object with the following structure:
{{
    "winner": <1, 2, or 0>,
    "reasoning": "Answer 1 is better because <your reasoning>."
}}

---Question---
{question}

---Answer 1---
{answer_1}

---Answer 2---
{answer_2}

Assess which answer is better according to the following measure:
{criterion_name}
"""

In [31]:
def evaluate_pair(question, ans1, ans2, criterion_name):
    """Envoie les deux réponses au LLM Juge et récupère le gagnant."""
    
    prompt = JUDGE_USER_TEMPLATE.format(
        criterion_def=CRITERIA[criterion_name],
        question=question,
        answer_1=ans1,
        answer_2=ans2,
        criterion_name=criterion_name
    )

    try:
        response = client.chat.completions.create(
            model="gpt-4-mini", # Le papier recommande un modèle puissant pour le juge [3]
            messages=[
                {"role": "system", "content": JUDGE_SYSTEM_PROMPT},
                {"role": "user", "content": prompt}
            ],
            response_format={"type": "json_object"}, # Force le JSON pour faciliter le parsing
            temperature=0 # On veut de la cohérence, même si on répète le test
        )
        
        result_text = response.choices.message.content
        result_json = json.loads(result_text)
        return result_json.get("winner"), result_json.get("reasoning")
        
    except Exception as e:
        print(f"Erreur lors de l'appel API: {e}")
        return 0, "Error"

In [32]:
def run_comparison(test_set, criterion="comprehensiveness", repetitions=5):
    """
    Compare GraphRAG vs RAG Classique sur un jeu de données.
    A = GraphRAG (Answer 1)
    B = RAG Classique (Answer 2)
    """
    question_scores = []

    print(f"--- Début de l'évaluation sur le critère : {criterion.upper()} ---")

    for i, item in enumerate(test_set):
        print(f"\nQuestion {i+1}: {item['question'][:50]}...")
        
        scores_for_this_question = []
        
        # Répéter l'évaluation 5 fois pour gérer la stochasticité du LLM
        for r in range(repetitions):
            winner, reasoning = evaluate_pair(
                item['question'], 
                item['answer_graphrag'], 
                item['answer_standard_rag'], 
                criterion
            )
            
            # Attribution des points selon la méthodologie [4]
            if winner == 1:
                score = 100  # GraphRAG gagne
            elif winner == 2:
                score = 0    # RAG Classique gagne
            else:
                score = 50   # Égalité
                
            scores_for_this_question.append(score)
            print(f"  Rep {r+1}: Gagnant={winner} (Score={score}) | Raison: {reasoning[:50]}...")

        # Calculer la moyenne pour CETTE question
        avg_question_score = statistics.mean(scores_for_this_question)
        question_scores.append(avg_question_score)
        print(f"  >> Moyenne pour Question {i+1}: {avg_question_score}")
        final_win_rate = statistics.mean(question_scores)
    
    print("\n" + "="*50)
    print(f"RÉSULTAT FINAL ({criterion})")
    print(f"GraphRAG Win Rate: {final_win_rate:.2f}%")
    print("="*50)
    
    if final_win_rate > 50:
        print("Conclusion: GraphRAG est globalement meilleur.")
    elif final_win_rate < 50:
        print("Conclusion: RAG Classique est globalement meilleur.")
    else:
        print("Conclusion: Égalité parfaite.")

In [33]:
with open('answer.json', 'r', encoding='utf-8') as f:
    data = json.load(f)